# Models

Models are simply classes which inherit from `pydantic.BaseModel` and define fields as annotated attributes.

Models share many similarities with Python's dataclasses, but have been designed with some subtle-yet-import differences that streamline certain workflows related to validation, serialization, and JSON schema generation.

In [7]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'Jane Doe'

user = User(id='123')

In this example, `user` is an instance of `User`. Initialization of the object will perform all parsing and validation.

In [9]:
assert user.id == 123
assert isinstance(user.id, int)
assert user.name == 'Jane Doe'
assert user.model_fields_set == {'id'}
assert user.model_dump() == {'id': 123, 'name': 'Jane Doe'}

## Model methods and properties

| methods or properties | description |
| ---- | ---- |
| `model_computed_fields` | a dictionaty of the computed fields of this model instance. |
| `model_construct()` | a class method for creating models without running validation. |
| `model_copy()` | returns a copy (by default, shallow copy) of the model.
| `model_dump()` | returns a dictionary of the model's fields and values |
| `model_dump_json()` | returns a JSON string representation of `model_dump()` |
| `model_extra` | get extra fields set during validation |
| `model_fields_set` | set of fields which were set when the model instance was initialized. |
| `model_json_schema` | returns a jsonable dictionary representing the model as JSON Schema. |
| `model_parametrized_name()` | |
| `model_validate()` | a utility for loading any object into a model. |
| `model_validate_json()` | a utility for validating the given JSON data against the Pydantic model |

In [15]:
from typing import List, Optional
from pydantic import BaseModel

class Foo(BaseModel):
    count: int
    size: Optional[float] = None

class Bar(BaseModel):
    apple: str = 'x'
    banana: str = 'y'

class Spam(BaseModel):
    foo: Foo
    bar: List[Bar]

m = Spam(foo={'count': 4}, bar=[{'apple': 'x1'}, {'apple': 'x2'}])
print(m)

print(m.model_dump())

foo=Foo(count=4, size=None) bar=[Bar(apple='x1', banana='y'), Bar(apple='x2', banana='y')]
{'foo': {'count': 4, 'size': None}, 'bar': [{'apple': 'x1', 'banana': 'y'}, {'apple': 'x2', 'banana': 'y'}]}


In [19]:
from pydantic import BaseModel, PydanticUserError

class Foo(BaseModel):
    x: 'Bar'

try:
    Foo.model_json_schema()
except PydanticUserError as e:
    print(e)

class Bar(BaseModel):
    pass

Foo.model_rebuild()
print(Foo.model_json_schema())

{'$defs': {'Bar': {'properties': {}, 'title': 'Bar', 'type': 'object'}}, 'properties': {'x': {'$ref': '#/$defs/Bar'}}, 'required': ['x'], 'title': 'Foo', 'type': 'object'}


In [ ]:
from typing import List
from pydantic import BaseModel, ConfigDict

class PetCls:
    def __init__(self, *, name: str, species: str):
        self.name = name
        self.species = species

class PersonCls:
    def __init__(self, *, name: str, age: float = None, pets: List[PetCls]):
        self.name = name
        self.age = age
        self.pets = pets

class Pet(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    name: str
    species: str

class Person(BaseModel):
    model_config = ConfigDict(from_attributes=True)
    name: str
    age: float = None
    pets: List[Ped]

bones = )et

# Fields

# JSON Schema

# Types

# Models

# Models

In [1]:
# Example
from datetime import datetime
from typing import Tuple

from pydantic import BaseModel


class Delivery(BaseModel):
    timestamp: datetime
    dimensions: Tuple[int, int]


m = Delivery(timestamp='2020-01-02T03:04:05Z', dimensions=['10', '20'])
print(repr(m.timestamp))
#> datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
print(m.dimensions)
#> (10, 20)

datetime.datetime(2020, 1, 2, 3, 4, 5, tzinfo=TzInfo(UTC))
(10, 20)


In [2]:
# Example
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int  
    name: str = 'John Doe'  
    signup_ts: datetime | None  
    tastes: dict[str, PositiveInt]  


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',  
    'tastes': {
        'wine': 9,
        b'cheese': 7,  
        'cabbage': '1',  
    },
}

user = User(**external_data)  

print(user.id)  
#> 123
print(user.model_dump())  
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""


123
{'id': 123, 'name': 'John Doe', 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1}}


"\n{\n    'id': 123,\n    'name': 'John Doe',\n    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),\n    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},\n}\n"

In [3]:
# Excample

# continuing the above example...

from pydantic import ValidationError


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}  

try:
    User(**external_data)  
except ValidationError as e:
    print(e.errors())
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """


[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'not an int', 'url': 'https://errors.pydantic.dev/2.5/v/int_parsing'}, {'type': 'missing', 'loc': ('signup_ts',), 'msg': 'Field required', 'input': {'id': 'not an int', 'tastes': {}}, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]


In [5]:
# Example

from typing import Annotated, Dict, List, Literal, Tuple
from annotated_types import Gt
from pydantic import BaseModel

class Fruit(BaseModel):
    name: str
    color: Literal['red', 'green']
    weight: Annotated[float, Gt(0)]
    bazam: Dict[str, List[Tuple[int, bool, float]]]

print(
    Fruit(
        name='Apple',
        color='red',
        weight=4.2,
        bazam={'foobar': [(1, True, 0.1)]},
    )
)

name='Apple' color='red' weight=4.2 bazam={'foobar': [(1, True, 0.1)]}


## Performance

Pydantic's core validation logic is implemented in separate package `pydantic-core`, where validation fro most types is implemented in Rust.

## Serialization

Pydantic provides functionality to seerialize model in three ways:

1. To a Python `dict` made up of the associated Python objects
2. To a Python `dict` made up only of "jsonable" types
3. To a JSON string

## JSON Schema

JSON Schema can be generated from any Pydantic schema.

## Strict mode and data coercion

By default, Pydantic is tolerant to common incorrect types and coerces data to the right type.

Pydantic also has `strict=True` mode, where types are not coerced and a validation error is raised unless the input data exactly amtches the schema or type hint.

## Dataclasses, TypedDicts, and more

Pydantic provides four ways to create schemas and perform validation and serialization:

1. `BaseModel` - Pydantic's own super class with many common utilities available via instance methods.
2. `pydantic.dataclasses.dataclass` - a wrapper around standard dataclasses which performs validation when a dataclass is initializedc.
3. `TypeAdapter` - a general way to adapt any type for validation and serialization.
4. `validate_call` - a decorator to perform validation when calling a function.

5. 